In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install rapidfuzz -qq

In [ ]:
import matplotlib.pyplot as plt
import base64 
from PIL import Image
import io
from rapidfuzz import process, fuzz
from tqdm import tqdm
import urllib

# Languages
Histograms of languages in each train tsv file
> 
Top 10 languages:
1. en
2. de
3. fr
4. en
5. ru
6. it
7. nl
8. pl
9. ja
10. uk

In [ ]:
'''
for i in range(5):
    filename = f'/kaggle/input/wikipedia-image-caption/train-0000{i}-of-00005.tsv'
    df = pd.read_csv(filename, sep='\t', usecols=['language'])
    plt.figure(figsize=[20,3])
    df["language"].value_counts().plot(kind="bar")
'''

# Preditions only from image URLs
- extract texts from the images urls
- find captions that matche the extracted texts.

## Updates (Sep 23)
- use `urllib.parse.unquote` to decode percent-encoded sequences into Unicode characters
- use `fuzz.token_set_ratio` as a scorer

In [ ]:
'''
df_test = pd.read_csv('/kaggle/input/wikipedia-image-caption/test.tsv', sep='\t')
df_test['txt_from_url'] = df_test['image_url'].apply(lambda x: x.split('/')[-1][:-4].replace('_', ' '))
df_captions = pd.read_csv('../input/wikipedia-image-caption/test_caption_list.csv')
captions = df_captions['caption_title_and_reference_description'].tolist()

preds_list = []
for ID, txt in tqdm(df_test[['id', 'txt_from_url']].values):
    result = process.extract(txt, captions, scorer=fuzz.ratio, processor=None, limit=5)
    for line in result:
        preds_list.append([ID, line[0]])
'''

In [ ]:
df_test = pd.read_csv('/kaggle/input/wikipedia-image-caption/test.tsv', sep='\t')
df_test['txt_from_url'] = df_test['image_url'].apply(lambda x: urllib.parse.unquote(x).split('/')[-1][:-4].replace('_', ' '))
df_captions = pd.read_csv('../input/wikipedia-image-caption/test_caption_list.csv')
captions = df_captions['caption_title_and_reference_description'].tolist()

In [ ]:
preds_list = []
for ID, txt in tqdm(df_test[['id', 'txt_from_url']].values):
    result = process.extract(txt, captions, scorer=fuzz.token_set_ratio, processor=None, limit=5)
    for line in result:
        preds_list.append([ID, line[0]])

In [ ]:
sub = pd.DataFrame(preds_list, columns=['id', 'caption_title_and_reference_description'])
sub.to_csv('submission.csv', index=None)